# Climate-related classification tasks on ChatGPT - using techniques for better prompt engineering
<hr>
<h3>In this Notebook we have used various techniques for better prompt engineering in order to get better results, including chaining of prompts, using different patterns for prompts like <i>"The Persona Pattern"</i> where which enables
the model to take a certain point of view or role, in our
case, a climate, sustainability, and environmental expert; <i>“The Fact Check List Pattern”</i>, which instructs the model
to output the most important points of a text and then use
those points as the input in a follow-up prompt and the
<i>“Reflection Pattern”</i> in which the model is asked to
explain the reasoning behind its response.</h3>
<h3>For that purpose, in this script we have created a function that connects to the OpenAI API using the API key and sends the data and the prompts provided to ChatGPT in batch, in chosen batch size. Additionally, a function that chains prompts was also created such that it receives the first prompt and forwards it as input in the second prompt.</h3>
<h3>In order to be able to run the scripts and the tasks, first the OpenAI key needs to be set.</h3>
<h3>Because this key is a secret and gives access to your OpenAI account, it should be hidden and not available in plain text to the public. It is advised to store these keys in files on your computer on on some cloud, like Google Drive where other people cannot access them and then open them in the Notebook and set the keys via variables, that way they can be protected from the public.</h3>
<h3>In our approach, we used text files on Google Drive to store the key and we open them in the Notebook, set the appropriate variable and then use the variable to set the key.</h3>
<hr>
<h3>To use this script, you need to set your OpenAI key, to do that, if you use the same approach as us, first you need to store your key in a file and store them on Google Drive and after that only the path to the file in which the key is stored needs to be changed and the script will work.</h3>
<h3>Alternative approaches include uploading your locally stored files to the Colab Notebook, using a GitHub repository or using alternative storage solutions.</h3>
<h3>On the following link you can find ways to deal with your files on various storage providers: <a href="https://neptune.ai/blog/google-colab-dealing-with-files">https://neptune.ai/blog/google-colab-dealing-with-files</a></h3>
<hr>
<h3>Each task is structured in its own Colab Notebook and in order to get the results for a task, first the appropriate keys must be set in the Notebook and after that the whole Notebook can just be run and the results will be displayed at the end of the section, either by collapsing the section and running the cells from the whole section at once or running each cell one by one. Some steps are optional, for example saving the results in a .csv file and may be skipped.</h3>

In [ ]:
#With the following commands, your Google Drive gets mounted to the Notebook at /content/drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai
!pip install tiktoken
import openai
import tiktoken
encoding = tiktoken.get_encoding("gpt2")
with open('Here put the path to your OpenAI API key', 'r') as file:
    key = file.readline()

#Alternatively, you can just insert your keys as plain text in the appropriate places, but this is not advised since your keys would be visible to anyone who has access to your Notebook
#For using other approaches, please visit the link provided in the description above that instructs use and import of files from other storage solutions

openai.api_key = key

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached openai-0.27.8-py3-none-any.whl (73 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.5 MB/s eta 0:00:00


In [ ]:
import json
import time
def batch_gpt(prompt,target_texts, batch_size):
    l = len(target_texts)
    size = int(l / batch_size) + 1
    size = batch_size
    text_list = np.array_split(target_texts, 3)
    print(f"Total records {l}, number of chunks = {size}")
    rez_keys = []
    rez_vals = []
    rez = []
    for i, texts in enumerate(text_list):
        text = "\n".join(texts)
        p = prompt + text
        print("prompt", p)
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": p}]
        )
        r = response["choices"][0]["message"]["content"]
        print(result, r)
        rez.append(r)
        try:
            dictData= json.loads(r)
            keys = list(dictData.keys())
            values = list(dictData.values())
            rez_keys += keys
            rez_vals += values
        except:
            print("error parsing"+r)
        print(f"i={i + 1}: shape:{len(rez_keys)}")
    return rez_keys, rez_vals, rez
def batch_gpt_len(prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = int(r[:1])
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys


def batch_gpt_chained(prompt,chained_prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r1 = response["choices"][0]["message"]["content"]
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": chained_prompt + text}]
            )
        except:
            print("error calling API, retrying...")
            time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = int(r[:1])
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys



def batch_gpt_resp_as_part_prompt(prompt,chained_prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    summarized_points = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r1 = response["choices"][0]["message"]["content"]
        summarized_points.append(r1)
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": chained_prompt + r1}]
            )
        except:
            print("error calling API, retrying...")
            time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = int(r[:1])
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys, summarized_points




def batch_gpt_verify_cot_prompts(start_prompt,end_prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = start_prompt + text + end_prompt
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            parts = r.split("\"")
            i = len(parts)-2
            b = parts[i]
            b = b.lower()
            if b.startswith("yes"):
              b = 1
            else:
              b = 0
        except:
            b = -1
            print("error parsing"+r)
        # print(i, len(p), "-", r, b, end=": ")
        print(f"{i}: \| response: {r} \| result: {b}\n\n")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys

<h1>Running the Climate Detection Task on ChatGPT</h1>
<h4>In the Climate Detection task, paragraphs are being classified whether they are climate-related or not.</h4>
<hr>
<h4>Classification classes:</h4>
<h4>0 - is not climate change-related</h4>
<h4>1 - is climate change-related</h4>
<hr>
<h4>First, the required library - datasets is loaded in order to be able to work with the dataset and the corresponding dataset is downloaded from HuggingFace and loaded into the dataset variable.</h4>

In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("climatebert/climate_detection")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.4 MB/s eta 0:00:00


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1300 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/climatebert___parquet/climatebert--climate_detection-eefa7af3c8031d26/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1300
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [ ]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 400
})

<h4>After that, the paragraphs and the labels are extracted from the dataset and are loaded into a Pandas DataFrame that allows easier manipulation with the data and better visualization of the data, with tables.</h4>

In [ ]:
data = []
for i in range(0,len(dataset['test']['text'])):
  data.append([dataset['test']['text'][i],dataset['test']['label'][i]])

print(data)

[['Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.', 1], ['Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.', 1], ['In 2019, the Company closed a series of transactions related to

In [ ]:
import pandas as pd
df = pd.DataFrame(data=data,columns=["text","label"])

In [ ]:
df

text  label
0    Sustainable strategy ‘red lines’ For our susta...      1
1    Verizon’s environmental, health and safety man...      1
2    In 2019, the Company closed a series of transa...      1
3    In December 2020, the AUC approved the Electri...      1
4    Finally, there is a reputational risk linked t...      1
..                                                 ...    ...
395  In 2020, Banco do Brasil Foundation celebrated...      1
396  Climate change is producing changes in weather...      1
397  A sound and certain regulatory and fiscal envi...      0
398  Across our global workforce, 20% of Gold Field...      0
399  The purchase price of these raw materials and ...      0

[400 rows x 2 columns]

<h4>The initial prompt that was sent to ChatGPT was the following: "You are the sustainability, environment, and climate change expert. Read the following text about sustainability, the environment, or climate change and analyze it as an expert:"</h4>
<h4>After the response was received, the response was then used as the input for the second prompt in which ChatGPT is instructed to perform the classification. The chained prompt was the following: "Now you need to classify the analyzed text whether the text is about sustainability, the environment, or climate change. Answer only with 1 if the text is sustainability, environment or climate change related or 0 if not:"</h4>
<hr>
<h4>The results were received both in a numerical - categorical and textual representation</h4>

In [ ]:
prompt = 'You are the sustainability, environment, and climate change expert. Read the following text about sustainability, the environment, or climate change and analyze it as an expert: \n\n'
chain_prompt = 'Now you need to classify the analyzed text whether the text is about sustainability, the environment, or climate change. Answer only with 1 if the text is sustainability, environment or climate change related or 0 if not: \n\n'
texts = df["text"].to_list()
rez, rez_keys = batch_gpt_chained(prompt,chain_prompt, texts, 1)

1 409 - 1 (sustainability related) 1: 2 867 - 1 (environment related) 1: 3 796 - 0 0: 4 1109 - 0 0: 5 599 - 1 (climate change related) 1: 6 568 - 1 (environment and sustainability related) 1: 7 565 - 1 (climate change related) 1: 8 613 - 1 (sustainability related) 1: 9 709 - 1 (Environment and Climate Change related) 1: 10 484 - 1 (climate change related) 1: 
11 773 - 0 0: error calling API, retrying...
error parsingAs an AI language model, I do not hold the expertise of a sustainability, environment, and climate change expert. However, the given text highlights the significance of climate change and its implications for the insurance industry. It indicates that businesses are acknowledging the importance of understanding and addressing climate-related risks and opportunities. This is crucial as climate change can have long-term effects on business operations, making it necessary for companies to integrate sustainable practices. Thus, it is encouraging to see businesses recognizing the

<h4>The predictions that are received, are stored in a variable, then they are stored both in numerical and textual representations in a Pandas DataFrame to further be compared and evaluated.</h4>

In [ ]:
rez

['1 (sustainability related)',
 '1 (environment related)',
 '0',
 '0',
 '1 (climate change related)',
 '1 (environment and sustainability related)',
 '1 (climate change related)',
 '1 (sustainability related)',
 '1 (Environment and Climate Change related)',
 '1 (climate change related)',
 '0',
 'As an AI language model, I do not hold the expertise of a sustainability, environment, and climate change expert. However, the given text highlights the significance of climate change and its implications for the insurance industry. It indicates that businesses are acknowledging the importance of understanding and addressing climate-related risks and opportunities. This is crucial as climate change can have long-term effects on business operations, making it necessary for companies to integrate sustainable practices. Thus, it is encouraging to see businesses recognizing the need for sustainability and taking proactive measures to manage climate change risks.',
 '1 (sustainability related)',
 '0

In [ ]:
rez_keys

[1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 -1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 -1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,

In [ ]:
df['gpt-label'] = rez_keys

In [ ]:
df['gpt-explanations'] = rez

In [ ]:
df

text  label  gpt-label  \
0    Sustainable strategy ‘red lines’ For our susta...      1          1   
1    Verizon’s environmental, health and safety man...      1          1   
2    In 2019, the Company closed a series of transa...      1          0   
3    In December 2020, the AUC approved the Electri...      1          0   
4    Finally, there is a reputational risk linked t...      1          1   
..                                                 ...    ...        ...   
395  In 2020, Banco do Brasil Foundation celebrated...      1          1   
396  Climate change is producing changes in weather...      1          1   
397  A sound and certain regulatory and fiscal envi...      0          0   
398  Across our global workforce, 20% of Gold Field...      0          0   
399  The purchase price of these raw materials and ...      0          0   

               gpt-explanations  
0    1 (sustainability related)  
1       1 (environment related)  
2                             0  
3                             0  
4    1 (climate change related)  
..                          ...  
395  1 (sustainability related)  
396  1 (climate change related)  
397                           0  
398                           0  
399                           0  

[400 rows x 4 columns]

<h4>The labels that couldn't be automatically mapped were mapped manually by observing their textual counterparts and seeing where they belong.</h4>

In [ ]:
df['gpt-label'].value_counts()

 1    288
 0    105
-1      7
Name: gpt-label, dtype: int64

In [ ]:
df[df['gpt-label']==-1]

text  label  gpt-label  \
11   Climate-related risks and opportunities have g...      1         -1   
22    To exclude the least energy efficient hydroc...      1         -1   
52   We also look at the way banks facilitate finan...      1         -1   
97   New products include our $1 per day electric v...      1         -1   
99   The scenarios developed by the IEA were used a...      1         -1   
301  Although climate change has not yet had a mate...      1         -1   
323  As we invest in Manufactured Capital to improv...      1         -1   

                                      gpt-explanations  
11   As an AI language model, I do not hold the exp...  
22                                   Climate change: 1  
52   As a sustainability, environment, and climate ...  
97   As a sustainability expert, I can see that AGL...  
99   As a sustainability, environment, and climate ...  
301  operations and across the financial sector, an...  
323       and Social capitals. 1 (Environment related)

In [ ]:
df['gpt-label'] = df['gpt-label'].replace(-1,1)

In [ ]:
df['gpt-label'].value_counts()

1    295
0    105
Name: gpt-label, dtype: int64

<h4>The DataFrame is also stored on Google Drive, for later viewing and analysis. This step can be skipped.</h4>

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_detection_chained_prompts.csv",index=False)

<h4>In the following section, the predicted labels are compared to the actual labels and the results are displayed.</h4>
<hr>
<h4>In the first row of the output, three metrics are displayed in the following order: <h6>(precision, recall, fscore, support - optional, may be none)</h6></h4>
<h4>In the second row, only the F1 Score is displayed, for better clarity.</h4>
<h4>In the third row the confusion matrix is displayed.</h4>
<h4>In the fourth row the whole classification report is displayed, with the metrics per class: precision, recall, f1 score and support; the accuracy, per class and overall and the macro and micro averages of each metric.</h4>

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.8163841807909604, 0.8828125, 0.8417930125247197, None)
0.8417930125247197
[[ 70  10]
 [ 35 285]]
              precision    recall  f1-score   support

           0       0.67      0.88      0.76        80
           1       0.97      0.89      0.93       320

    accuracy                           0.89       400
   macro avg       0.82      0.88      0.84       400
weighted avg       0.91      0.89      0.89       400

